# Gravitee.io setup

## Env Init and MongoDB setup

In [ ]:
. activate

First we'll create a secret for our MongoDB database

In [ ]:
cat > config.yml <<EOF
apiVersion: v1
kind: Secret
metadata:
  name: gravitee-mongodb
type: Opaque
data:
  database: gravitee
stringData:
  username: gravitee
  password: "Sup3rStr0ngPassw0rd"
EOF

In [ ]:
kubectl apply -f config.yml

We need now to create a user account associated with the MongoDB database we'll be using

In [ ]:
# Admin credential
MONGODB_ADMIN_USER=root
MONGODB_ADMIN_PASS=$(kubectl get secret my-mongodb -o jsonpath="{.data.mongodb-root-password}" | base64 --decode)
# DB connection settings
MONGODB_HOST=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].metadata.name}.{.items[0].metadata.namespace}.svc.cluster.local")
MONGODB_PORT=$(kubectl get service -l release=my-mongodb -o jsonpath="{.items[0].spec.ports[0].port}")
# Gravitee credential
MONGODB_DB=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.database}")
MONGODB_GRAVITEE_USER=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.username}" | base64 --decode)
MONGODB_GRAVITEE_PASS=$(kubectl get secret gravitee-mongodb -o jsonpath="{.data.password}" | base64 --decode)
echo Host: $MONGODB_HOST, Port: $MONGODB_PORT

In [ ]:
# https://docs.mongodb.com/manual/reference/built-in-roles/#database-user-roles
cat > config.js <<EOF
db = db.getSiblingDB("$MONGODB_DB")
db.dropUser("$MONGODB_GRAVITEE_USER")
db.createUser({"user": "$MONGODB_GRAVITEE_USER", "pwd": "$MONGODB_GRAVITEE_PASS", "roles": ["dbOwner"]})
EOF

In [ ]:
cat config.js | kubectl run my-mongodb-client --rm -i --restart='Never' --image bitnami/mongodb --command -- mongo --host $MONGODB_HOST:$MONGODB_PORT -u $MONGODB_ADMIN_USER -p $MONGODB_ADMIN_PASS

## ElasticSearch setup

We need to gather some ElasticSearch configuration values

In [ ]:
ES_PROTOCOL=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].spec.ports[0].name}")
ES_PORT=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].spec.ports[0].port}")
ES_HOST=$(kubectl get service -l release=my-elasticsearch -o jsonpath="{.items[0].metadata.name}.{.items[0].metadata.namespace}.svc.cluster.local")

In [ ]:
echo Protocol: $ES_PROTOCOL, Host: $ES_HOST, Port: $ES_PORT

## Gravitee installation

The first step is to build the helm [chart](https://github.com/gravitee-io/gravitee-kubernetes/tree/master/gravitee).

In [ ]:
curl -OL https://github.com/gravitee-io/gravitee-kubernetes/archive/master.zip

In [ ]:
unzip -ou master.zip

In [ ]:
helm package gravitee-kubernetes-master/gravitee/ # build chart

In [ ]:
INGRESS_HOSTS='{}'

In [ ]:
helm del --purge gravitee
helm install --name gravitee gravitee-1.22.0.tgz \
--set mongo.rsEnabled=false \
--set mongo.dbhost=$MONGODB_GRAVITEE_USER:$MONGODB_GRAVITEE_PASS@$MONGODB_HOST \
--set mongo.dbport=$MONGODB_PORT \
--set es.protocol=$ES_PROTOCOL \
--set es.host=$ES_HOST \
--set es.port=$ES_PORT \
--set ui.ingress.hosts=$INGRESS_HOSTS \
--set api.ingress.hosts=$INGRESS_HOSTS \
--set gateway.ingress.hosts=$INGRESS_HOSTS

Check that the pods are up and running

In [ ]:
kubectl get pods -l release=gravitee